# Profiling Brute Force vs Forward Algorithm

In [1]:
from coefficients import A, B, PI
import datetime as dt

from markov import hot, cold, STATES, Observation
from fractions import Fraction

from algorithms import p_observations_given_states, p_states, forward

In [2]:
from itertools import product

In [3]:
LENGTH = 3


In [4]:
def brute_force(observations: list[Observation]) -> Fraction:
    weather_sequences = list(product(STATES, repeat=len(observations)))

    return sum(
        [
            p_observations_given_states(B, states, observations)
            * p_states(PI, A, states)
            for states in weather_sequences
        ]
    )


def optimized(observations: list[Observation]) -> Fraction:
    alphas = list(forward(PI, A, B, observations))
    return sum(alphas[-1].values())

In [5]:
brute_force([3, 1, 3])

Fraction(14281, 500000)

In [6]:
optimized([3, 1, 3])

Fraction(14281, 500000)

Now let's profile with a longer sequence 

In [7]:
def compare(observations: list[Observation]):
    start = dt.datetime.now()
    po_brute = brute_force(observations)
    d1 = dt.datetime.now() - start

    start = dt.datetime.now()
    po_optimized = optimized(observations)
    d2 = dt.datetime.now() - start

    assert po_brute == po_optimized

    print(
        f"""
    Brute force: {d1}. 
    Optimized:   {d2}. 
    Ratio:       {d1.total_seconds() / d2.total_seconds():.4f}"""
    )

In [8]:
compare([3, 1, 3])


    Brute force: 0:00:00.000067. 
    Optimized:   0:00:00.000024. 
    Ratio:       2.7917


In [9]:
compare([3, 1, 3,1,1])


    Brute force: 0:00:00.000290. 
    Optimized:   0:00:00.000040. 
    Ratio:       7.2500


In [10]:
compare([3, 1, 3,1,1,2,3,1,1,2,3,1,3,1,2,3])


    Brute force: 0:00:01.330804. 
    Optimized:   0:00:00.000165. 
    Ratio:       8065.4788


Brute force approach is something like $O(n^i)$, where $N$ is number of states and $i$ is number of steps.

Optimized approach is linear in $i$